# Hybrid



In [4]:
%pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


## Import Model

Load model with pickle

In [2]:
import pickle

svd = pickle.load(open('./model/svd.pkl', 'rb'))

In [3]:
# create a test user and item
uid = str(20901)
iid = str(232507)

# get prediction
prediction = svd.predict(uid, iid)
prediction.est

2.2744071056785615

In [12]:
import numpy as np
import pandas as pd

cosine_sim = np.loadtxt('./cs.csv', delimiter=',')
dfCS = pd.read_csv('./cs_cv.csv', header=None)

In [14]:
cosine_sim

array([[1188.,  178.,  176., ...,   68.,  148.,  151.],
       [ 178.,  531.,  371., ...,   80.,   50.,  117.],
       [ 176.,  371.,  548., ...,   90.,   46.,  142.],
       ...,
       [  68.,   80.,   90., ...,  191.,   29.,   47.],
       [ 148.,   50.,   46., ...,   29.,  140.,   40.],
       [ 151.,  117.,  142., ...,   47.,   40.,  493.]])

In [10]:
dfCS.head()

,0,1,2,3,4,5,6,7,8,9,...,3857,3858,3859,3860,3861,3862,3863,3864,3865,3866
0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,3857.0,3858.0,3859.0,3860.0,3861.0,3862.0,3863.0,3864.0,3865.0,3866.0
1,1188.0,178.0,176.0,189.0,141.0,184.0,20.0,237.0,90.0,39.0,...,168.0,79.0,175.0,137.0,130.0,122.0,197.0,68.0,148.0,151.0
2,178.0,531.0,371.0,52.0,88.0,94.0,30.0,127.0,69.0,29.0,...,107.0,72.0,121.0,117.0,89.0,78.0,76.0,80.0,50.0,117.0
3,176.0,371.0,548.0,65.0,95.0,128.0,38.0,134.0,69.0,17.0,...,119.0,78.0,118.0,110.0,99.0,100.0,90.0,90.0,46.0,142.0
4,189.0,52.0,65.0,338.0,54.0,78.0,17.0,122.0,63.0,8.0,...,58.0,33.0,135.0,88.0,47.0,49.0,79.0,26.0,45.0,125.0


In [16]:
dfCampgrounds = pd.read_csv('./REC_Content_Facility.csv')

In [17]:
indices = pd.Series(dfCampgrounds.index, dfCampgrounds['FacilityName'])

In [18]:
indices

FacilityName
Burro Creek Campground              0
Cold Brook Campground               1
Cottonwood Springs Campground       2
Steel Creek Campground              3
Chinamen's Campground               4
                                 ... 
Cloverleaf Campground (WA)       3862
Hawk Creek Campground            3863
Jones Bay Campground             3864
TEN MILE CAMPGROUND              3865
Juniper Campground Group Site    3866
Length: 3867, dtype: int64

In [35]:
def hybrid_recommender(user, name):
    # get on facility name
    idx = indices[name]

    # calculate similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # sort scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # take ten scores
    # omitting the first entry because it is the same item 
    sim_scores = sim_scores[1:11]

    # map scores to campgrounds
    campground_indices = [i[0] for i in sim_scores]

    # build a dataframe with results
    # include faciltity info and similarity score
    dfResults = dfCampgrounds[['FacilityID', 'FacilityName']].iloc[campground_indices]
    dfResults['Score'] = dfResults['FacilityID'].apply(lambda x: svd.predict(str(user), str(x)).est)

    # sort results in descending order
    dfResults = dfResults.sort_values('Score', ascending=False)

    # return results
    return dfResults

In [36]:
hybrid_recommender(uid, 'ASSATEAGUE ISLAND NATIONAL SEASHORE CAMPGROUND')

,FacilityID,FacilityName,Score
3672,261700,SANTA ROSA ISLAND BACKCOUNTRY BEACH CAMPING,2.274407
3599,253730,Cumberland Island National Seashore Camping Pe...,2.274407
3310,249981,Voyageurs National Park Camping Permits,2.274407
1094,232501,SAN MIGUEL ISLAND,2.274407
1092,232499,SANTA CRUZ DEL NORTE BACKCOUNTRY,2.274407
1090,232497,SANTA ROSA ISLAND,2.274407
1091,232498,SANTA CRUZ SCORPION,2.274407
1908,233359,Point Reyes National Seashore Campground,2.274407
3657,258851,Pine Springs Campground,2.274407
3298,247663,Signal Mountain Campground,2.274407


In [34]:
svd.predict(uid, str(247663)).est

2.2744071056785615